In [1]:
from tqdm import tqdm
import polars as pl
import json

In [2]:
krs = pl.read_parquet("/home/stef/enz_rxn_data/data/processed/pathway/known_reactions.parquet")
krs[2000:2005]

id,smarts,enzymes,reverse,db_ids
i32,str,list[str],i32,list[str]
2035,"""*C1CCC2C3CCC4C(C)(CCC(O)C4(C)C…","[""Q70VZ7"", ""Q96PD6"", … ""Q80W94""]",2039,"[""RHEA:60089""]"
2036,"""*C1CCC2C3CCC4C(C)(CO)C(O)CCC4(…","[""P52704""]",2031,"[""RHEA:60057""]"
2037,"""*C1CCC2C3CCC4C(C)(CO)C(O)CCC4(…","[""P52704""]",2029,"[""RHEA:60054""]"
2038,"""*C1CCC2C3CCC4C(C)C(=O)CCC4(C)C…","[""Q8BMJ2"", ""Q9P2J5"", ""Q5R614""]",2034,"[""RHEA:60086""]"
2039,"""*C1CCC2C3CCC4C(C)C(=O)CCC4(C)C…","[""Q70VZ7"", ""Q96PD6"", … ""Q80W94""]",2035,"[""RHEA:60090""]"


In [3]:
with open("/home/stef/enz_rxn_data/data/raw/pathway/sprhea_240310_v3_mapped_no_subunits.json", "r") as f:
    old = json.load(f)

In [4]:
old_rxns_w_enz = [v for v in old.values() if len(v['enzymes']) > 0]

In [5]:
new_rxns_w_enz = krs.filter(pl.col("enzymes").list.len() > 0)

In [6]:
old_enz = set()
old_pairs = set()
old_proper_enz = set()
old_proper_pairs = set()
for k, rxn in tqdm(old.items()):
    for enz in rxn['enzymes']:
        old_enz.add(enz['uniprot_id'])
        old_pairs.add((k, enz['uniprot_id']))

        if enz['existence'] in ["Evidence at protein level", "Evidence at transcript level"]:
            old_proper_enz.add(enz['uniprot_id'])
            old_proper_pairs.add((k, enz['uniprot_id']))

100%|██████████| 18954/18954 [00:00<00:00, 48330.41it/s]


In [7]:
kes = pl.read_parquet("/home/stef/enz_rxn_data/data/processed/pathway/known_enzymes.parquet")
kes.head()

id,sequence,existence,reviewed,ec,organism,name
str,str,enum,enum,str,str,str
"""A0A009IHW8""","""MSLEQKKGADIISKILQIQNSIGKTTSPST…","""Evidence at protein level""","""reviewed""","""3.2.2.-; 3.2.2.6""","""Acinetobacter baumannii (strai…","""2' cyclic ADP-D-ribose synthas…"
"""A0A059TC02""","""MRSVSGQVVCVTGAGGFIASWLVKILLEKG…","""Evidence at protein level""","""reviewed""","""1.2.1.44""","""Petunia hybrida (Petunia)""","""Cinnamoyl-CoA reductase 1 (Ph-…"
"""A0A061I403""","""MPMASVIAVAEPKWISVWGRFLWLTLLSMA…","""Evidence at protein level""","""reviewed""","""2.7.7.108; 3.1.4.-""","""Cricetulus griseus (Chinese ha…","""Protein adenylyltransferase FI…"
"""A0A067XGX8""","""MALTATATTRGGSALPNSCLQTPKFQSLQK…","""Evidence at transcript level""","""reviewed""","""2.5.1.54""","""Petunia hybrida (Petunia)""","""Phospho-2-dehydro-3-deoxyhepto…"
"""A0A067XH53""","""MALSTNSTTSSLLPKTPLVQQPLLKNASLP…","""Evidence at transcript level""","""reviewed""","""2.5.1.54""","""Petunia hybrida (Petunia)""","""Phospho-2-dehydro-3-deoxyhepto…"


In [8]:
kes.filter(pl.col("existence").is_in(["Evidence at protein level", "Evidence at transcript level"]))

id,sequence,existence,reviewed,ec,organism,name
str,str,enum,enum,str,str,str
"""A0A009IHW8""","""MSLEQKKGADIISKILQIQNSIGKTTSPST…","""Evidence at protein level""","""reviewed""","""3.2.2.-; 3.2.2.6""","""Acinetobacter baumannii (strai…","""2' cyclic ADP-D-ribose synthas…"
"""A0A059TC02""","""MRSVSGQVVCVTGAGGFIASWLVKILLEKG…","""Evidence at protein level""","""reviewed""","""1.2.1.44""","""Petunia hybrida (Petunia)""","""Cinnamoyl-CoA reductase 1 (Ph-…"
"""A0A061I403""","""MPMASVIAVAEPKWISVWGRFLWLTLLSMA…","""Evidence at protein level""","""reviewed""","""2.7.7.108; 3.1.4.-""","""Cricetulus griseus (Chinese ha…","""Protein adenylyltransferase FI…"
"""A0A067XGX8""","""MALTATATTRGGSALPNSCLQTPKFQSLQK…","""Evidence at transcript level""","""reviewed""","""2.5.1.54""","""Petunia hybrida (Petunia)""","""Phospho-2-dehydro-3-deoxyhepto…"
"""A0A067XH53""","""MALSTNSTTSSLLPKTPLVQQPLLKNASLP…","""Evidence at transcript level""","""reviewed""","""2.5.1.54""","""Petunia hybrida (Petunia)""","""Phospho-2-dehydro-3-deoxyhepto…"
…,…,…,…,…,…,…
"""P80580""","""MKLYSFFNXRA""","""Evidence at protein level""","""reviewed""","""5.2.1.4""","""Klebsiella pneumoniae""","""Maleylpyruvate isomerase (EC 5…"
"""P83329""","""XXFENFLNANXAYVNLD""","""Evidence at protein level""","""reviewed""","""4.2.1.1""","""Streptococcus thermophilus""","""Putative carbonic anhydrase (E…"
"""P83855""","""GVYMEIGRCRXEAXRRRKEAV""","""Evidence at protein level""","""reviewed""","""4.6.1.1""","""Mus musculus (Mouse)""","""Putative sperm adenylate cycla…"


In [9]:
enzymes = set()
pairs = set()
proper_pairs = set()
proper_enzymes = set()
evidence_dict = dict(zip(kes["id"], kes["existence"]))
for row in tqdm(krs.iter_rows(named=True), total=len(krs)):
    rxn = row['smarts']
    for enz in row['enzymes']:
        enzymes.add(enz)
        evidence = evidence_dict.get(enz, "Unknown")
        
        if evidence in ["Evidence at protein level", "Evidence at transcript level"]:
            proper_pairs.add((rxn, enz))
            proper_enzymes.add(enz)
        
        pairs.add((rxn, enz))

len(pairs), len(proper_pairs)

100%|██████████| 28142/28142 [00:00<00:00, 51224.30it/s]


(518578, 164968)

In [10]:
print("total reacitions")
print(f"old: {len(old_rxns_w_enz)} vs new: {krs.shape[0]}")
print("\nreactions with enzymes")
print(f"old: {len(old_rxns_w_enz)} vs new: {new_rxns_w_enz.shape[0]}")

total reacitions
old: 18954 vs new: 28142

reactions with enzymes
old: 18954 vs new: 21034


In [11]:
print("number of enzymes")
print(f"old: {len(old_enz)} vs new: {len(enzymes)}")
print("\nnumber of proper enzymes")
print(f"old: {len(old_proper_enz)} vs new: {len(proper_enzymes)}")
print("\nnumber of pairs")
print(f"old: {len(old_pairs)} vs new: {len(pairs)}")
print("\nnumber of proper pairs")
print(f"old: {len(old_proper_pairs)} vs new: {len(proper_pairs)}")

number of enzymes
old: 180013 vs new: 188937

number of proper enzymes
old: 36531 vs new: 41238

number of pairs
old: 485570 vs new: 518578

number of proper pairs
old: 140234 vs new: 164968


In [12]:
kcs = pl.read_parquet("/home/stef/enz_rxn_data/data/processed/pathway/known_compounds.parquet")
kcs.tail()

id,smiles,name,chebi_id,n_atoms
i32,str,str,str,i32
10589,"""c1ccncc1""",""" pyridine""","""CHEBI:16227""",6
10590,"""c1cn[nH]c1""",""" pyrazole""","""CHEBI:17241""",5
10591,"""c1cncc(C2CCCN2)c1""",""" (S)-nornicotine""","""CHEBI:190184""",11
10592,"""c1ncc2c(n1)=NCCN=2""",""" 6,7-dihydropteridine""","""CHEBI:30156""",10
10593,"""c1ncc2c(n1)NCCN2""",""" 5,6,7,8-tetrahydropteridine""","""CHEBI:28889""",10


In [13]:
len(kcs)

10594